In [ ]:
from components.PdfMemberExtractor import PdfMemberExtractor
import pandas as pd


In [3]:
extractor = PdfMemberExtractor()
df_pdf = extractor.extract(pdf_path="../data/ERC-2025-StG-panel-members.pdf")
df_pdf['ERC-Date'] = df_pdf['ERC-Date'].astype(int)
erc_date = int(df_pdf['ERC-Date'].unique()[0])
display(df_pdf)


----------------------------------------

           Chair                    Member ERC-Date
0   Satoru Iwata            Luigi Ambrosio     2025
1   Satoru Iwata           Carolina Araujo     2025
2   Satoru Iwata              Boris Aronov     2025
3   Satoru Iwata        Stéphane Boucheron     2025
4   Satoru Iwata            Alice Guionnet     2025
5   Satoru Iwata            Gitta Kutyniok     2025
6   Satoru Iwata        Elon Lindenstrauss     2025
7   Satoru Iwata            Ilaria Perugia     2025
8   Satoru Iwata      Laura Maria Sangalli     2025
9   Satoru Iwata            Thomas Scanlon     2025
10  Satoru Iwata  Carola-Bibiane Schönlieb     2025
11  Satoru Iwata           Fabio Toninelli     2025
12  Satoru Iwata            Karen Vogtmann     2025
13  Satoru Iwata           Katrin Wendland     2025

----------------------------------------


----------------------------------------

                Chair                  Member ERC-Date
14  Giulia Zanderighi  Eric Arnold B

,Chair,Member,ERC-Date
0,Satoru Iwata,Luigi Ambrosio,2025
1,Satoru Iwata,Carolina Araujo,2025
2,Satoru Iwata,Boris Aronov,2025
3,Satoru Iwata,Stéphane Boucheron,2025
4,Satoru Iwata,Alice Guionnet,2025
...,...,...,...
442,Helena Wulff,Kevin Smets,2025
443,Helena Wulff,Sharika Thiranagama,2025
444,Helena Wulff,Rob van der Laarse,2025
445,Helena Wulff,Lorenz Korn,2025


array(['PE1', 'PE2', 'PE3', 'PE4', 'PE6', 'PE7', 'PE8', 'PE9', 'PE10',
       'PE11', 'LS1', 'LS2', 'LS3', 'LS5', 'LS7', 'SH3', 'SH4', 'SH5',
       'SH6', 'SH7', 'SH8'], dtype=object)

In [46]:
# Read panel-members-excel.xls
excel_path = "../data/panel-members-excel.xls"

df_excel = pd.read_excel(excel_path, engine='openpyxl')

# split column year at "," and copy the row for each year
df_excel = df_excel.assign(year=df_excel['year'].astype(str).str.split(',')).explode('year')

# strip whitespace und entferne leere/ungültige Einträge (nur 4-stellige Jahreszahlen)
df_excel['year'] = df_excel['year'].str.strip()
df_excel = df_excel[df_excel['year'].str.match(r'^\s*\d{4}\s*$', na=False)].copy()

# year in int konvertieren
df_excel['year'] = df_excel['year'].astype(int)



# display only the rows where year == erc_date
df_excel = df_excel[df_excel['year'] == erc_date]

# strip whitespace from Name column
df_excel['Name'] = df_excel['Name'].str.strip()



display(df_excel.head())

,Name,funding_scheme,review_panel,year
61,Deborah Lawlor,SyG,LS,2025
89,Hilleke Hulshoff Pol,SyG,LS,2025
112,Sharon Tooze,SyG,LS,2025
244,Ramesh Pillai,CoG,LS1,2025
274,Rumiana Dimova,StG,LS1,2025


In [47]:


review_panels = df_pdf['review_panel'].unique()
df_merged_back = pd.DataFrame()
for panel in review_panels:
    df_panel = df_pdf[df_pdf['review_panel'] == panel]
    
    df_merged_panel = pd.merge(df_panel, df_excel, left_on='Fullname', right_on='Name', how='left', indicator=True)
    
    # if review_panel_x != review_panel_y, then use review_panel_y
    df_merged_panel['review_panel'] = df_merged_panel.apply(
        lambda row: row['review_panel_y'] if row['review_panel_x'] != row['review_panel_y'] and pd.notna(row['review_panel_y']) else row['review_panel_x'],
        axis=1
    )

    # if the chair person has a different panel, set all members to chair's panel
    chair_member = df_merged_panel[df_merged_panel['Type'] == 'Chair']
    if not chair_member.empty:
        chair_panel = chair_member['review_panel'].values[0]
        # Setze alle Mitglieder auf das Panel des Chairs, wenn es unterschiedlich ist
        if chair_panel != panel:
            df_merged_panel['review_panel'] = chair_panel

   
    
     
    
    # merge back to df_merged_back
    df_merged_back = pd.concat([df_merged_back, df_merged_panel], ignore_index=True)

# review_panels = df_merged_back['review_panel'].unique()
for panel in review_panels:
    df_panel = df_merged_back[df_merged_back['review_panel'] == panel]
    display(df_panel[['Fullname', 'review_panel_x', 'review_panel_y', 'review_panel']])

,Fullname,review_panel_x,review_panel_y,review_panel
0,Satoru Iwata,PE1,PE1,PE1
1,Luigi Ambrosio,PE1,PE1,PE1
2,Carolina Araujo,PE1,PE1,PE1
3,Boris Aronov,PE1,PE1,PE1
4,Stéphane Boucheron,PE1,PE1,PE1
5,Alice Guionnet,PE1,PE1,PE1
6,Gitta Kutyniok,PE1,PE1,PE1
7,Elon Lindenstrauss,PE1,PE1,PE1
8,Ilaria Perugia,PE1,PE1,PE1
9,Laura Maria Sangalli,PE1,NaN,PE1


,Fullname,review_panel_x,review_panel_y,review_panel
15,Giulia Zanderighi,PE2,PE2,PE2
16,Eric Arnold Bergshoeff,PE2,NaN,PE2
17,Frédéric Chevy,PE2,PE2,PE2
18,Brigitte Cros,PE2,PE2,PE2
19,Péter Domokos,PE2,PE2,PE2
20,Aurél Gábris,PE2,PE2,PE2
21,Mariusz Gajda,PE2,PE2,PE2
22,Oldřich Kepka,PE2,PE2,PE2
23,Zohar Komargodski,PE2,PE2,PE2
24,Blaženka Melić,PE2,PE2,PE2


,Fullname,review_panel_x,review_panel_y,review_panel
31,Daniele Brida,PE3,PE3,PE3
32,M. Isabel Alonso,PE3,NaN,PE3
33,Sergej Demokritov,PE3,PE3,PE3
34,Daan Frenkel,PE3,PE3,PE3
35,Maia Vergniory,PE3,PE3,PE3
36,Juan Garrahan,PE3,PE3,PE3
37,Julie Grollier,PE3,PE3,PE3
38,Pertti Hakonen,PE3,PE3,PE3
39,Paul Janmey,PE3,PE3,PE3
40,Despina Louca,PE3,PE3,PE3


,Fullname,review_panel_x,review_panel_y,review_panel
46,Dónal Leech,PE4,PE4,PE4
47,Sara Bals,PE4,PE4,PE4
48,Perdita Barran,PE4,PE4,PE4
49,Daniel Bélanger,PE4,PE4,PE4
50,Ria Broer-Braam,PE4,PE4,PE4
51,Michele Ceriotti,PE4,PE4,PE4
52,Ilaria Ciofini,PE4,PE4,PE4
53,Gaël de Paëpe,PE4,PE4,PE4
54,Mariangela Di Donato,PE4,PE4,PE4
55,Michael Lyons,PE4,PE4,PE4


,Fullname,review_panel_x,review_panel_y,review_panel


...

,Fullname,review_panel_x,review_panel_y,review_panel
367,Gabriella Vigliocco,SH4,SH4,SH4
368,Heleen Slagter,SH4,SH4,SH4
369,Gregoire Borst,SH4,SH4,SH4
370,Cristina Botella,SH4,SH4,SH4
371,Ann Copestake,SH4,SH4,SH4
372,Elisabetta Crocetti,SH4,SH4,SH4
373,Daniel Everett,SH4,SH4,SH4
374,Ingmar Franken,SH4,SH4,SH4
375,Didier Grandjean,SH4,SH4,SH4
376,Silvia Luraghi,SH4,SH4,SH4


,Fullname,review_panel_x,review_panel_y,review_panel
385,Maciej Eder,SH5,SH5,SH5
386,Hanoch Ben-Yami,SH5,SH5,SH5
387,Elisabetta Cattanei,SH5,SH5,SH5
388,Göran Collste,SH5,SH5,SH5
389,Robert Hawley,SH5,SH5,SH5
390,Shazia Jagot,SH5,SH5,SH5
391,Thomas Y Levin,SH5,SH5,SH5
392,Katarzyna Marciniak,SH5,SH5,SH5
393,Carmen Meinert,SH5,SH5,SH5
394,Parimal Patil,SH5,SH5,SH5


,Fullname,review_panel_x,review_panel_y,review_panel
402,László Kontler,SH6,SH6,SH6
403,Rosa Maria Albert,SH6,SH6,SH6
404,Miguel Bandeira Jerónimo,SH6,SH6,SH6
405,James H. Barrett,SH6,SH6,SH6
406,Amy Bogaard,SH6,SH6,SH6
407,Christian Bonah,SH6,SH6,SH6
408,Vinita Damodaran,SH6,SH6,SH6
409,Sven Dupré,SH6,SH6,SH6
410,Dagmar Ellerbrock,SH6,SH6,SH6
411,Marcella Frangipane,SH6,SH6,SH6


,Fullname,review_panel_x,review_panel_y,review_panel
420,Tiit Tammaru,SH7,SH7,SH7
421,Thomas Elmqvist,SH7,SH7,SH7
422,Jack Ahern,SH7,SH7,SH7
423,Anne Laure Amilhat Szary,SH7,NaN,SH7
424,Itzhak Benenson,SH7,SH7,SH7
425,Marta Bivand Erdal,SH7,SH7,SH7
426,Sarah Brauner-Otto,SH7,SH7,SH7
427,Eric Clark,SH7,SH7,SH7
428,Ferne Edwards,SH7,SH7,SH7
429,Eran Feitelson,SH7,SH7,SH7


,Fullname,review_panel_x,review_panel_y,review_panel
439,Helena Wulff,SH8,SH8,SH8
440,Stefano Allievi,SH8,SH8,SH8
441,Bart Barendregt,SH8,SH8,SH8
442,Elaine Chew,SH8,SH8,SH8
443,Karmen Franinović,SH8,SH8,SH8
444,Heather Horst,SH8,SH8,SH8
445,Thomas Da Costa Kaufmann,SH8,SH8,SH8
446,Susan Levine,SH8,SH8,SH8
447,Chandana Mathur,SH8,SH8,SH8
448,Giedrė Mickūnaitė,SH8,SH8,SH8


In [6]:
df_merged = pd.merge(df_pdf, df_excel, left_on='Fullname', right_on='Name', how='left', indicator=True)
display(df_merged)


,Domain,Subdomain,Forename,Lastname,Fullname,Type,ERC-Date,review_panel_x,Name,funding_scheme,review_panel_y,year,_merge
0,PE,Mathematics (PE1),Satoru,Iwata,Satoru Iwata,Chair,2025,PE1,Satoru Iwata,StG,PE1,2025.0,both
1,PE,Mathematics (PE1),Luigi,Ambrosio,Luigi Ambrosio,Member,2025,PE1,Luigi Ambrosio,StG,PE1,2025.0,both
2,PE,Mathematics (PE1),Carolina,Araujo,Carolina Araujo,Member,2025,PE1,Carolina Araujo,StG,PE1,2025.0,both
3,PE,Mathematics (PE1),Boris,Aronov,Boris Aronov,Member,2025,PE1,Boris Aronov,StG,PE1,2025.0,both
4,PE,Mathematics (PE1),Stéphane,Boucheron,Stéphane Boucheron,Member,2025,PE1,Stéphane Boucheron,StG,PE1,2025.0,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...
472,LS,Studies of Cultures and Arts (SH8),Kevin,Smets,Kevin Smets,Member,2025,SH8,Kevin Smets,StG,SH8,2025.0,both
473,LS,Studies of Cultures and Arts (SH8),Sharika,Thiranagama,Sharika Thiranagama,Member,2025,SH8,Sharika Thiranagama,StG,SH8,2025.0,both
474,LS,Studies of Cultures and Arts (SH8),Rob,van der Laarse,Rob van der Laarse,Member,2025,SH8,NaN,NaN,NaN,NaN,left_only
475,LS,Studies of Cultures and Arts (SH8),Lorenz,Korn,Lorenz Korn,Member,2025,SH8,NaN,NaN,NaN,NaN,left_only


In [ ]:
df_merged = df_merged.loc[:, ['Forename', "Lastname", "Fullname", "Type", "year", "funding_scheme"]]


display(df_merged.info())
display(df_merged)

print(df_merged["funding_scheme"].value_counts())